In [33]:
import psycopg2
import pandas as pd
import sqlalchemy

In [65]:
conn = psycopg2.connect(host="127.0.0.1", port="5432", database="Adventureworks", 
                            user="postgres", password="postgres")
cur = conn.cursor()

In [42]:
cur = conn.cursor()

cur.execute('''
           create table if not exists  customer 
            (customer_id int,
             store_id int,
             first_name varchar,
             last_name varchar)
            ''')


cur.execute('''
           insert into   customer
            values(1,1,'Mike','Mike');
            ''')


cur.execute('''
           select* from customer
        
            ''')

results = cur.fetchall()
pd.DataFrame(results, columns=['customer_id', 'store_id', 'first_name', 'last_name'])

,customer_id,store_id,first_name,last_name
0,1,1,Mike,Mike


In [36]:
cur = conn.cursor()

cur.execute('''
           create or replace function first_function()
            returns setof customer
            as $$
            select * from customer;
            $$
            language sql
            ''')


cur.execute('''
           select * from first_function();
            ''')

results = cur.fetchall()
pd.DataFrame(results, columns=['customer_id', 'store_id', 'first_name', 'last_name'])

,customer_id,store_id,first_name,last_name
0,1,1,Mike,Mike


### вызов функции из функции


In [9]:

cur.execute('''
            Create or replace function call_function_in_function()
            returns text
            as $$
                select first_name from  first_function() limit 1;
            $$ 
            language sql;
            ''')

cur.execute('''
           select * from public.call_function_in_function();
            ''')
results = cur.fetchall()
pd.DataFrame(results, columns=['first_function'])

,first_function
0,Mike


### Разрешение функций с переменными параметрами

In [11]:
cur.execute('''
            CREATE or REPLACE FUNCTION public.variadic_example(VARIADIC numeric[]) 
                RETURNS int
                  LANGUAGE sql AS 'SELECT 1';

            ''')

cur.execute('''
            SELECT public.variadic_example(0),
               public.variadic_example(0.0),
               public.variadic_example(VARIADIC array[0.0]);

            ''')

results = cur.fetchall()


In [14]:
pd.DataFrame(results, columns=['variadic_example_1', 'variadic_example_2',
                               'variadic_example_3'])

,variadic_example_1,variadic_example_2,variadic_example_3
0,1,1,1


### Функция преобразования строки 


In [17]:
cur.execute('''
            create or replace function lower_or_upper
            (last_name text, first_name text, flag int default 0)
            returns text
            as $$
              select case $3
                            when 0 then INITCAP ($1 || ' ' || $2)
                            when 1 then upper($1 || ' ' || $2)
                            when 2 then lower($1 || ' ' || $2)
                            else $1 || ' ' || $2
                            end;
            $$ Language sql;

            ''')

cur.execute('''
           select 
           lower_or_upper('СидрОрОВ', 'ИвАн',1);
            ''')

results = cur.fetchall()
pd.DataFrame(results, columns=['lower_or_upper'])

,lower_or_upper
0,СИДРОРОВ ИВАН


In [18]:
cur.execute('''
           select lower_or_upper(last_name=> 'СидрОрОВ', first_name=>'ИвАн');
            ''')

results = cur.fetchall()
pd.DataFrame(results, columns=['lower_or_upper'])

,lower_or_upper
0,Сидроров Иван


In [19]:
cur.execute('''
           select lower_or_upper('СидрОрОВ', 'ИвАн', flag=>2);
            ''')

results = cur.fetchall()
pd.DataFrame(results, columns=['lower_or_upper'])

,lower_or_upper
0,сидроров иван


### Функция возвращает несколько значений

In [20]:
cur.execute('''
           create or replace function return_setof_int(max_i integer) 
            returns setof int as
            $$
            declare
                i integer;
            begin
              for i in 1 .. max_i
              loop
             return next i;
            end loop;
            end;
            $$ language plpgsql; 
            ''')

cur.execute('''
           select return_setof_int(5); 
            ''')

results = cur.fetchall()
pd.DataFrame(results, columns=['return_setof_int'])

,return_setof_int
0,1
1,2
2,3
3,4
4,5


### пример переменной inout


In [26]:
cur.execute('''
           create or replace function return_inout(inout result1 int, out result2 int)
            as $$
            begin
            result2 = result1;  result1= 10*result1;
            return;
            end
            $$ language plpgsql;  
            ''')


cur.execute('''
           select return_inout(9);
            ''')

results = cur.fetchall()
pd.DataFrame(results, columns=['return_inout'])

,return_inout
0,"(90,9)"


### вывод таблицы customer через функцию

In [37]:
cur.execute('''
            Create or replace function return_customer(out store int) 
            as
            --do
            $$
            begin
                select store_id into store from public.customer limit 1;
                return;
            end 
            $$ language plpgsql; 
            ''')


cur.execute('''
           select * from  public.return_customer();
            ''')

results = cur.fetchall()
pd.DataFrame(results, columns=['return_customer'])

,return_customer
0,1


### транзакции в функции


In [43]:
cur.execute('''
           Create or replace function fn_transaction() returns setof customer as
            $$
                update customer set first_name= 'Mike' where customer_id = 1;
                select * from customer where customer_id = 1;
            $$ language sql;

            ''')

cur.execute('''
            select * from fn_transaction();
            ''')

results = cur.fetchall()
pd.DataFrame(results, columns=['customer_id', 'store_id', 'first_name', 'last_name'])

,customer_id,store_id,first_name,last_name
0,1,1,Mike,Mike


### использование переменной VARIADIC массив с переменным набором аргументов
### Выдаёт ряд целых чисел от start до stop с шагом 1
### generate_subscripts — это «функция, возвращающая множество», которая будет возвращать несколько строк при ее вызове. 

In [48]:
cur.execute('''
            CREATE or replace FUNCTION mleast(VARIADIC arr numeric[]) 
            RETURNS numeric AS $$
                SELECT min($1[i]) FROM generate_subscripts($1, 1) g(i);
            $$ LANGUAGE SQL;

            ''')

cur.execute('''
            SELECT mleast(9, -1, 5, 4);
            ''')

results = cur.fetchall()
pd.DataFrame(results, columns=['mleast'])

,mleast
0,-1


 ### Еще один вывод таблицы customer через функцию
 ### цикл for

In [50]:
cur.execute('''
            create or replace function select_customer() 
             returns table(f_name text, l_name text) as 
             $$
             begin  
                 for f_name, l_name in select first_name, last_name from customer
                 loop
                    return next;
                 end loop;
             end
             $$ language plpgsql;
            ''')


cur.execute('''
             select * from select_customer();
            ''')

results = cur.fetchall()
pd.DataFrame(results, columns=['f_name', 'l_name'])

,f_name,l_name
0,Mike,Mike


 ### функция возвращает сведения о времени последнего процесса vacuum, analyze


In [52]:
cur.execute('''
             create or replace function sql_last_v_a() 
            returns setof record as
            'select schemaname,relname, last_vacuum, last_analyze
            from pg_stat_all_tables
            order by schemaname, relname;' 
            language sql; 
            ''')

cur.execute('''
            select sql_last_v_a();
            ''')

results = cur.fetchall()
pd.DataFrame(results, columns=['sql_last_v_a'])

,sql_last_v_a
0,"(humanresources,department,,)"
1,"(humanresources,employee,,)"
2,"(humanresources,employeedepartmenthistory,,)"
3,"(humanresources,employeepayhistory,,)"
4,"(humanresources,jobcandidate,,)"
...,...
226,"(sales,salesterritoryhistory,,)"
227,"(sales,shoppingcartitem,,)"
228,"(sales,specialoffer,,)"
229,"(sales,specialofferproduct,,)"


### cистемные функции https://postgrespro.ru/docs/postgrespro/13/functions-info

In [54]:
cur.execute('''
            select current_catalog; 
            --имя текущей базы данных (в стандарте SQL она называется «каталогом»)
            ''')


results = cur.fetchall()
pd.DataFrame(results, columns=['current_catalog'])

,current_catalog
0,Adventureworks


In [55]:
cur.execute('''   
        select current_database();
        --имя текущей базы данных
            ''')


results = cur.fetchall()
pd.DataFrame(results, columns=['current_database'])

,current_database
0,Adventureworks


In [56]:
cur.execute('''   
        select current_user;
            ''')


results = cur.fetchall()
pd.DataFrame(results, columns=['current_user'])

,current_user
0,postgres


In [57]:
cur.execute('''   
        select version();
            ''')


results = cur.fetchall()
pd.DataFrame(results, columns=['version'])

,version
0,PostgreSQL 15.1 (Debian 15.1-1.pgdg110+1) on a...


### CTE FUNC

In [68]:
query = f'''

     select *
     FROM sales.countryregioncurrency
  
'''
df = pd.read_sql_query(query, conn)
df.head()

/Users/denis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,countryregioncode,currencycode,modifieddate
0,AE,AED,2014-02-08 10:17:21.510
1,AR,ARS,2014-02-08 10:17:21.510
2,AT,ATS,2014-02-08 10:17:21.510
3,AT,EUR,2008-04-30 00:00:00.000
4,AU,AUD,2014-02-08 10:17:21.510


In [73]:
cur.execute('''   
        Create or replace function public.cte_in_function()
            returns text
            --returns table(full_address text) 
            as $$
            WITH countryregioncurrency_cte AS (
                select  countryregioncode || ' ' ||
                currencycode || ' ' || modifieddate as countryregioncurrency
                FROM sales.countryregioncurrency
               )
            SELECT countryregioncurrency
            FROM countryregioncurrency_cte
            $$ Language sql;
            ''')

cur.execute('''   
        select public.cte_in_function();
            ''')

results = cur.fetchall()
pd.DataFrame(results, columns=['countryregioncurrency'])

,countryregioncurrency
0,AE AED 2014-02-08 10:17:21.51


In [64]:
conn.rollback()
conn.close()